In [7]:
from dotenv import load_dotenv
import os
from pydantic import BaseModel, Field
from pydantic_ai import Agent
from typing import List

load_dotenv()

True

Data structure / BaseModel

In [8]:
class PainPoint(BaseModel):
    description: str = Field(description="What's not working for the user. Be specific about the problem, not the solution.")
    impact: str = Field(description="Cost in time, money, or confidence. Include quotes with numbers when available.")
    quote: str = Field(description="User's exact words revealing this pain")
    
class JobToBeDone(BaseModel):
    functional_job: str = Field(description="What task are they trying to complete?")
    emotional_job: str = Field(description="What they care about and how do they want to feel?")
    context: str = Field(description="When/where does this happen?")
    quote: str

class Workaround(BaseModel):
    what_they_do: str = Field(description="The workaround(s) they've created")
    why_needed: str = Field(description="What problem does the workaround solve?")
    cost: str = Field(description="Time/effort/risks this workaround takes")
    quote: str

class DesiredOutcome(BaseModel):
    outcome: str = Field(description="What do they really want?")
    current_gap: str = Field(description="Why can't they achieve this now?")
    quote: str

class BehavioralSignal(BaseModel):
    observation: str = Field(description="What implicit things did they say/do that was revealing?")
    what_it_reveals: str = Field(description="The underlying need or belief")
    quote: str

class InterviewInsights(BaseModel):
    pain_points: List[PainPoint]
    jobs_to_be_done: List[JobToBeDone]
    workarounds: List[Workaround]
    desired_outcomes: List[DesiredOutcome]
    behavioral_signals: List[BehavioralSignal]

Create AI agent that extracts insights:

In [9]:
multi_insight_agent = Agent(
    model='openai:gpt-4o-mini',
    output_type=InterviewInsights,
    system_prompt="""You are an expert product researcher analyzing user interviews using established frameworks (Jobs-to-be-Done, continuous discovery practices).
    
    Extract ALL insights from the interview:
    - Pain points: Problems causing time waste, costs, uncertainty, frustration
    - Jobs-to-be-done: What they're trying to accomplish (functional + emotional goals)
    - Workarounds: Current hacks/solutions they've created
    - Desired outcomes: What do they really want
    - Behavioral signals: Implicit patterns (what they do that reveals underlying needs)
    
    Always include exact quotes as evidence. Be thorough - one interview may have multiple insights of each type."""
)

Sample Interview:

Extract data models from the sample interview:

In [12]:
def load_transcript(filepath: str) -> str:
       """Load interview transcript from file"""
       with open(filepath, 'r') as f:
           return f.read()
   
   # Load and analyze
transcript = load_transcript('data/interviews/mock_interview.txt')
result = await multi_insight_agent.run(transcript)
insights = result.output

In [15]:
print("\nPAIN POINTS:")
for i, pp in enumerate(insights.pain_points, 1):
    print(f"\n{i}. {pp.description}")
    print(f"   Impact: {pp.impact}")
    print(f"   Quote: {pp.quote}")

print("\n\nJOBS-TO-BE-DONE:")
for i, job in enumerate(insights.jobs_to_be_done, 1):
    print(f"\n{i}. Functional: {job.functional_job}")
    print(f"   Emotional: {job.emotional_job}")
    print(f"   Context: {job.context}")

print("\n\nWORKAROUNDS:")
for i, w in enumerate(insights.workarounds, 1):
    print(f"\n{i}. {w.what_they_do}")
    print(f"   Why: {w.why_needed}")
    print(f"   Cost: {w.cost}")

print("\n\nDESIRED OUTCOMES:")
for i, outcome in enumerate(insights.desired_outcomes, 1):
    print(f"\n{i}. {outcome.outcome}")
    print(f"   Gap: {outcome.current_gap}")

print("\n\nBEHAVIORAL SIGNALS:")
for i, signal in enumerate(insights.behavioral_signals, 1):
    print(f"\n{i}. {signal.observation}")
    print(f"   Reveals: {signal.what_it_reveals}")


PAIN POINTS:

1. Encountering a large number of unexpected fields in the API response, leading to confusion.
   Impact: Wasted time and effort in understanding data fields—defining which ones to use takes longer than necessary.
   Quote: I ended up dumping the response into a JSON viewer and going field by field.

2. Inconsistent API response formats across different endpoints, which complicates onboarding for new developers.
   Impact: Increased training time and reduced onboarding efficiency.
   Quote: Right now, some endpoints return arrays, others wrap everything in an object with metadata.

3. Lack of observability in the API, making it difficult to detect issues until they become significant problems.
   Impact: Increased response times to problems; potential customer dissatisfaction due to undetected errors.
   Quote: If something goes wrong, I only notice when logs blow up or when a customer reports it.

4. No real testing sandbox, forcing reliance on potentially harmful testi